To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!


<a href="https://github.com/meta-llama/synthetic-data-kit"><img src="https://raw.githubusercontent.com/unslothai/notebooks/refs/heads/main/assets/meta%20round%20logo.png" width="137"></a>
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
    !pip install synthetic-data-kit==0.0.3
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1
    !pip install "git+https://github.com/AyushExel/synthetic-data-kit.git@feature/multimodal-lance-vqa"


In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

## Primary Goal
Our goal is to make Llama 3.2 3B understand the "Byte Latent Transformer: Patches Scale Better Than Tokens" [research paper](https://ai.meta.com/research/publications/byte-latent-transformer-patches-scale-better-than-tokens/) that was published in December 2024.

We'll use https://github.com/meta-llama/synthetic-data-kit to generate question and answer pairs **fully locally** which will be used for finetuning Llama 3.2 3B!

Check if it succeeded:

In [1]:
!API_ENDPOINT_KEY="LLM|704426635437672|3nFowHkWPXPZWYaepVCJC0Z3GMw" synthetic-data-kit system-check

Loading config from: /Users/ayushchaurasia/Documents/synth-data-kit/synthetic-data-kit/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /Users/ayushchaurasia/Documents/synth-data-kit/synthetic-data-kit/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /Users/ayushchaurasia/Documents/synth-data-kit/synthetic-data-kit/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Environment variable check:
API_ENDPOINT_KEY: Present
get_llm_provider returning: api-endpoint
API_ENDPOINT_KEY environment variable: Found
API key source: Environment variable
⠏ Checking API endpoint access.....INFO:httpx:HTTP Request: GET https://api.llama.com/v1/models "HTTP/1.1 200 OK"
 API endpoint access confirmed
Using custom API base: https://api.llama.com/v1
Default model: Llama-4-Maverick-17B-128E-Instruct-FP8
⠏ Checking API endpoint access....


## Document Parsing (PDF, CSV, HTML etc.)
Now, let's take the Byte Latent Transformer: Patches Scale Better Than Tokens research paper found at https://arxiv.org/abs/2412.09871 and covert it to Q&A pairs in order to finetune Llama 3.2!

In [4]:
!pip install playwright PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.4 MB/s eta 0:00:00


In [ ]:
!pip install pylance fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.9/425.9 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.3 MB/s eta 0:00:00


In [2]:
# Byte Latent Transformer: Patches Scale Better Than Tokens paper in HTML format
!synthetic-data-kit \
    ingest --multimodal "rl-guide.pdf" \
    --name "lora"


Loading config from: /Users/ayushchaurasia/Documents/synth-data-kit/synthetic-data-kit/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /Users/ayushchaurasia/Documents/synth-data-kit/synthetic-data-kit/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /Users/ayushchaurasia/Documents/synth-data-kit/synthetic-data-kit/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
⠙ Processing rl-guide.pdf...[2025-07-18T19:49:41Z WARN  lance::dataset::write::insert] No existing dataset at data/parsed/lora.lance, it will be created
⠹ Processing rl-guide.pdf...
✅ Text successfully extracted to data/parsed/lora.lance


### Peek at the parsed multimodal data in lance format

In [3]:
import lance

ds = lance.dataset("data/parsed/lora.lance")
print(ds.count_rows())
ds.to_table().to_pandas()

24


,text,image
0,"1\nText-to-speech, all model types & full fine...",None
1,"2\nThat's it! There are intricacies on what ""g...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
2,3\nAnother example is imagine you are given th...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
3,4\nOpenAI popularized the concept of RLHF\n (R...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
4,4\nOpenAI popularized the concept of RLHF\n (R...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
5,4\nOpenAI popularized the concept of RLHF\n (R...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
6,"5\nThe clip(..., 1-e, 1+e) term is used to for...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
7,6\nThis means GRPO is extremely efficient. Pre...,None
8,"7\nFor example for ""What is 2+2?"" we sample 4 ...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
9,"7\nFor example for ""What is 2+2?"" we sample 4 ...",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...


We see around 37 chunks of data. We now call synthetic-data-kit to create some pairs of data for 3 of our chunks.

You can process more chunks, but it'll be much slower!

Using `--num-pairs` will generate **approximately** that many QA pairs. However it might be shorter or longer depending on the `max_seq_length` of the loaded up model. So if you specify 100, you might only get 10 since the model's max sequence length is capped.

In [1]:

!API_ENDPOINT_KEY="LLM|704426635437672|3nFowHkWPXPZWYaepVCJC0Z3GMw" synthetic-data-kit create "data/parsed/lora.lance" --num-pairs 25 --type "vqa"

Loading config from: /Users/ayushchaurasia/Documents/synth-data-kit/synthetic-data-kit/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /Users/ayushchaurasia/Documents/synth-data-kit/synthetic-data-kit/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /Users/ayushchaurasia/Documents/synth-data-kit/synthetic-data-kit/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
get_llm_provider returning: api-endpoint
🔗 Using api-endpoint provider
Loading config from: 
/Users/ayushchaurasia/Documents/synth-data-kit/synthetic-data-kit/synthetic_data
_kit/config.yaml
Config has LLM provider set to: api-endpointed/lora.lance...
API_ENDPOINT_KEY from environment: Foundparsed/lora.lance...
Using API key: From env varnt from data/parsed/lora.lance...
Using API base URL: https://api.llama.com/v1ed/lora.lance...
L Using api-endpoint provider from data/parsed/lora.lance...
Loading config 

Optionally, you can clean up the data via pruning "bad" or low quality examples and convert the rest to JSON format for finetuning!

In [ ]:
for filename in [
                  "data/generated/reasoning_qa_multimodal.json",
                  "data/generated/lora_qa_multimodal.json",
                  "data/generated/deepseek-local_qa_multimodal.json"
                  ]:
  !API_ENDPOINT_KEY="LLM|704426635437672|3nFowHkWPXPZWYaepVCJC0Z3GMw" synthetic-data-kit \
      curate --threshold 5.0 {filename}


Loading config from: /usr/local/lib/python3.11/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.11/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.11/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
get_llm_provider returning: api-endpoint
🔗 Using api-endpoint provider
⠋ Cleaning content from data/generated/reasoning_qa_multimodal.json...
❌ Error: [Errno 2] No such file or directory: 
'data/generated/reasoning_qa_multimodal.json'
Loading config from: /usr/local/lib/python3.11/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.11/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.11/dist-packages/synthetic_data_kit/

We now convert the generated datasets into QA formats so we can load it for finetuning:

In [ ]:
for filename in [
    "data/cleaned/deepseek-local_qa_multimodal_cleaned.json",
    "data/cleaned/lora_qa_multimodal_cleaned.json",
    "data/cleaned/reasoning_qa_multimodal_cleaned.json"]:
  !synthetic-data-kit \
        save-as {filename} -f ft --storage lance

Loading config from: /usr/local/lib/python3.11/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.11/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loading config from: /usr/local/lib/python3.11/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
INFO:datasets:PyTorch version 2.6.0+cu124 available.
INFO:datasets:Polars version 1.21.0 available.
INFO:datasets:Duckdb version 1.2.2 available.
INFO:datasets:TensorFlow version 2.18.0 available.
INFO:datasets:JAX version 0.5.2 available.
⠋ Converting data/cleaned/deepseek-local_qa_multimodal_cleaned.json to ft format
with lance storage...
❌ Error: [Errno 2] No such file or directory: 
'data/cleaned/deepseek-local_qa_multimodal_cleaned.json'
Loading config from: /usr/local/lib/python3.11/dist-packages/synthetic_data_kit/config.yaml
Config has LLM provider set to: api-endpoint
Loadi

Let's load up the data and see what the synthetic data looks like!

In [ ]:
from datasets import Dataset
import pandas as pd
import lance
final_filenames = [
 "data/final/deepseek-local_qa_multimodal_cleaned_ft.lance",
 "data/final/lora_qa_multimodal_cleaned_ft.lance",
 "data/final/reasoning_qa_multimodal_cleaned_ft.lance"
]

conversations = pd.concat([
    lance.dataset(filename).to_table().to_pandas() for filename in final_filenames
]).reset_index(drop = True)

dataset = Dataset.from_pandas(conversations)

ValueError: Dataset at path content/data/final/deepseek-local_qa_multimodal_cleaned_ft.lance was not found: Not found: content/data/final/deepseek-local_qa_multimodal_cleaned_ft.lance/_versions, /home/runner/work/lance/lance/rust/lance-table/src/io/commit.rs:328:23, /home/runner/work/lance/lance/rust/lance/src/dataset/builder.rs:329:35

In [ ]:
dataset[11]

In [ ]:
dataset[20]

In [ ]:
dataset[300]

Finally free vLLM process to save memory and to allow for finetuning!

### Fine-tuning Synthetic Dataset with Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
    # Qwen3 new models
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    # Other very popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the `Llama-3.2` format for conversation style finetunes. The chat template renders conversations like below: (Cutting Knowledge Date is by default there!)

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 01 May 2025

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is 1+1?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

2<|eot_id|>
```

In [ ]:
dataset[0]

In [ ]:
import json

def formatting_prompts_func(examples):
    # Parse the JSON string in 'messages' into a Python list of dictionaries
    convos = [json.loads(message_str) for message_str in examples["messages"]]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return { "text" : texts, }

# Get our previous dataset and format it:
dataset = dataset.map(formatting_prompts_func, batched = True,)

See result of the first row:

In [ ]:
dataset[0]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! Use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [ ]:
messages = [
    {"role": "user", "content": "What model types are supported for text-to-speech?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer,
                   max_new_tokens = 256, temperature = 0.1)

The model learns about the research paper!!

In [ ]:
messages = [
    {"role": "user", "content": "What is the BitsType for IQ1_S??"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer,
                   max_new_tokens = 256, temperature = 0.1)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
messages = [
    {"role": "user", "content": "What is so special about BLT's tokenization?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer,
                   max_new_tokens = 256, temperature = 0.1)

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: # Change to True to upload finetune
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: # Change to True to upload finetune
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: # Change to True to upload finetune
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
if False: # Change to True to upload finetune
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # Change to True to upload finetune
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # Change to True to upload finetune
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
